<a href="https://colab.research.google.com/github/Gdearmascasta/Liberias-data/blob/main/Taller_groopby_Tabla_dinamica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
# Establecer semilla para reproducibilidad
np.random.seed(2025)

# Número de registros
n = 30

# Listas de valores posibles
ciudades = ["Cartagena", "Bogotá", "Medellín"]
categorias = ["A", "B", "C"]
clientes = ["Alpha", "Beta", "Gamma", "Delta", "Epsilon"]

# Generar fechas: 30 semanas a partir del 2025-01-01
fechas = pd.date_range("2025-01-01", periods=n, freq="7D")

# Crear el DataFrame
df_taller = pd.DataFrame({
    "Ciudad": np.random.choice(ciudades, size=n),
    "Categoría": np.random.choice(categorias, size=n),
    "Cliente": np.random.choice(clientes, size=n),
    "Ventas": np.random.randint(500, 5000, size=n),
    "Descuento": np.round(np.random.uniform(0.0, 0.25, size=n), 2),
    "Fecha": fechas
})

print("Dataset generado:")
print(df_taller.head(10))  # Mostrar primeras 10 filas para verificación
print("\nDescripción del dataset:")
print("El dataset simula un registro de ventas de contenedores realizado en diferentes ciudades, con distintos clientes y categorías de producto.")
print("Incluye:")
print("- Ciudad: ciudad donde se realiza la venta.")
print("- Categoría: tipo o línea de producto.")
print("- Cliente: comprador registrado en la transacción.")
print("- Ventas: valor monetario de cada venta en pesos.")
print("- Descuento: porcentaje de descuento aplicado en la venta.")
print("- Fecha: día en que se realizó la operación.")
print(f"Total de registros: {len(df_taller)}")

# Ejercicio 1 – Agrupación múltiple con métricas personalizadas
print("\n" + "="*50)
print("Ejercicio 1 – Agrupación múltiple con métricas personalizadas")
print("="*50)

# Agrupar por Ciudad y Categoría, calcular métricas
grouped = df_taller.groupby(['Ciudad', 'Categoría']).agg({
    'Ventas': ['sum', 'count'],
    'Descuento': 'mean'
}).round(2)

# Renombrar columnas para claridad
grouped.columns = ['Total_Ventas', 'Num_Transacciones', 'Promedio_Descuento']
grouped = grouped.reset_index()

print("Resultados del groupby:")
print(grouped)

# Pregunta: ¿En qué ciudad y con qué tipo de contenedor se venden más unidades, y dónde se aplican más descuentos?
max_transacciones = grouped.loc[grouped['Num_Transacciones'].idxmax()]
max_descuento = grouped.loc[grouped['Promedio_Descuento'].idxmax()]

print("\nConclusiones:")
print(f"- La combinación con más unidades vendidas es {max_transacciones['Ciudad']} - {max_transacciones['Categoría']} con {max_transacciones['Num_Transacciones']} transacciones.")
print(f"- La combinación con mayor promedio de descuento es {max_descuento['Ciudad']} - {max_descuento['Categoría']} con {max_descuento['Promedio_Descuento']*100:.1f}% de descuento promedio.")

# Ejercicio 2 – Transformaciones con comparación interna
print("\n" + "="*50)
print("Ejercicio 2 – Transformaciones con comparación interna")
print("="*50)

# Calcular el promedio de ventas por ciudad
promedio_por_ciudad = df_taller.groupby('Ciudad')['Ventas'].transform('mean')

# Agregar columna Dif_vs_media
df_taller['Dif_vs_media'] = df_taller['Ventas'] - promedio_por_ciudad

print("DataFrame con nueva columna Dif_vs_media:")
print(df_taller[['Ciudad', 'Ventas', 'Dif_vs_media']].head(10))

# Pregunta: ¿Qué ciudades tienen ventas que se destacan mucho por encima del promedio y cuáles están más bajas de lo normal?
estadisticas_ciudad = df_taller.groupby('Ciudad')['Dif_vs_media'].agg(['mean', 'std', 'min', 'max']).round(2)

print("\nEstadísticas de diferencias por ciudad:")
print(estadisticas_ciudad)

print("\nConclusiones:")
for ciudad in estadisticas_ciudad.index:
    mean_diff = estadisticas_ciudad.loc[ciudad, 'mean']
    max_diff = estadisticas_ciudad.loc[ciudad, 'max']
    min_diff = estadisticas_ciudad.loc[ciudad, 'min']
    if mean_diff > 0:
        print(f"- {ciudad}: Las ventas tienden a estar por encima del promedio (diferencia media: {mean_diff}). La venta máxima está {max_diff} por encima.")
    else:
        print(f"- {ciudad}: Las ventas tienden a estar por debajo del promedio (diferencia media: {mean_diff}). La venta mínima está {min_diff} por debajo.")
    if max_diff > 1000:
        print(f"  - Tiene ventas que se destacan mucho por encima (máx: {max_diff}).")
    if min_diff < -1000:
        print(f"  - Tiene ventas que están muy por debajo (mín: {min_diff}).")

# Ejercicio 3 – Tablas dinámicas básicas
print("\n" + "="*50)
print("Ejercicio 3 – Tablas dinámicas básicas")
print("="*50)

# Crear pivot table
pivot_basic = pd.pivot_table(df_taller, values='Ventas', index='Ciudad', columns='Categoría', aggfunc='sum', fill_value=0)

print("Pivot table básica:")
print(pivot_basic)

# Pregunta: ¿Cuáles son los tipos de contenedores que más se venden en cada ciudad?
print("\nConclusiones:")
for ciudad in pivot_basic.index:
    max_categoria = pivot_basic.loc[ciudad].idxmax()
    max_ventas = pivot_basic.loc[ciudad].max()
    print(f"- En {ciudad}, el tipo de contenedor que más se vende es la Categoría {max_categoria} con ventas totales de {max_ventas}.")

# Ejercicio 4 – KPIs múltiples en pivot
print("\n" + "="*50)
print("Ejercicio 4 – KPIs múltiples en pivot")
print("="*50)

# Crear pivot table con múltiples métricas
pivot_kpi = pd.pivot_table(df_taller, values='Ventas', index='Ciudad', aggfunc=['sum', 'mean', 'max'])

# Renombrar columnas
pivot_kpi.columns = ['Total_Ventas', 'Promedio_Ventas', 'Max_Venta']
pivot_kpi = pivot_kpi.round(2)

print("Pivot table con KPIs múltiples:")
print(pivot_kpi)

# Pregunta: ¿Qué ciudad logra el promedio de ventas más alto y cuál tuvo la venta más grande registrada?
ciudad_max_promedio = pivot_kpi['Promedio_Ventas'].idxmax()
max_promedio = pivot_kpi['Promedio_Ventas'].max()

ciudad_max_venta = pivot_kpi['Max_Venta'].idxmax()
max_venta = pivot_kpi['Max_Venta'].max()

print("\nConclusiones:")
print(f"- La ciudad con el promedio de ventas más alto es {ciudad_max_promedio} con un promedio de {max_promedio}.")
print(f"- La ciudad con la venta más grande registrada es {ciudad_max_venta} con una venta máxima de {max_venta}.")

# Ejercicio 5 – Ventas mensuales por cliente
print("\n" + "="*50)
print("Ejercicio 5 – Ventas mensuales por cliente")
print("="*50)

# Crear columna de mes
df_taller['Mes'] = df_taller['Fecha'].dt.to_period('M')

# Pivot table usando pd.pivot_table
pivot_mensual = pd.pivot_table(df_taller, values='Ventas', index='Cliente', columns='Mes', aggfunc='sum', fill_value=0)

print("Pivot table de ventas mensuales por cliente:")
print(pivot_mensual)

# Bonus: Repetir usando groupby(...).sum().unstack(fill_value=0)
groupby_mensual = df_taller.groupby(['Cliente', 'Mes'])['Ventas'].sum().unstack(fill_value=0)

print("\nUsando groupby().sum().unstack():")
print(groupby_mensual)

# Verificar si son iguales
son_iguales = pivot_mensual.equals(groupby_mensual)
print(f"\n¿Los resultados son iguales? {son_iguales}")

# Pregunta: ¿Qué clientes compran seguido todos los meses y cuáles compran de manera más irregular?
print("\nConclusiones:")
for cliente in pivot_mensual.index:
    ventas_mensuales = pivot_mensual.loc[cliente]
    meses_con_venta = (ventas_mensuales > 0).sum()
    total_meses = len(ventas_mensuales)
    consistencia = meses_con_venta / total_meses

    if consistencia == 1.0:
        print(f"- {cliente}: Compra seguido todos los meses (consistencia: 100%).")
    elif consistencia >= 0.5:
        print(f"- {cliente}: Compra de manera moderadamente regular (consistencia: {consistencia*100:.1f}%).")
    else:
        print(f"- {cliente}: Compra de manera irregular (consistencia: {consistencia*100:.1f}%).")

Dataset generado:
      Ciudad Categoría  Cliente  Ventas  Descuento      Fecha
0   Medellín         B     Beta    2538       0.06 2025-01-01
1   Medellín         C    Gamma    2603       0.09 2025-01-08
2   Medellín         B     Beta    3251       0.04 2025-01-15
3   Medellín         C    Alpha    3611       0.03 2025-01-22
4  Cartagena         A    Gamma    1024       0.02 2025-01-29
5  Cartagena         A  Epsilon    3995       0.04 2025-02-05
6  Cartagena         C    Alpha    4798       0.04 2025-02-12
7  Cartagena         C    Delta    4272       0.01 2025-02-19
8   Medellín         B    Gamma    1879       0.18 2025-02-26
9  Cartagena         B    Alpha    3465       0.17 2025-03-05

Descripción del dataset:
El dataset simula un registro de ventas de contenedores realizado en diferentes ciudades, con distintos clientes y categorías de producto.
Incluye:
- Ciudad: ciudad donde se realiza la venta.
- Categoría: tipo o línea de producto.
- Cliente: comprador registrado en la trans